In [56]:
from __future__ import division
import itertools
from random import randint, seed
from sage.all import gcd, inverse_mod

In [191]:
def new_mochila(N=10, jump=10, sd=0):
    seed(sd)
    diff = [randint(1,jump) for k in range(N)]
    mochila = [diff[0]]
    for i in range(1,len(diff)):
        new_val = sum(mochila) + diff[i]
        mochila.append(new_val)
    return mochila

def split2len(s, n=16):
    def _f(s, n):
        while s:
            yield s[:n]
            s = s[n:]
    return list(_f(s, n))

def text_to_bytes(text, n=0):
    largo = len(text)
    if n > 0:
        text = text + ' '*(n - (largo % n))
    text = text.decode('utf8')
    text_e = ''.join(['{0:016b}'.format(ord(c)) for c in text])
    return text_e

def bytes_to_text(binary):
    return ''.join([unichr(int(ch,2)) for ch in split2len(binary)])

def select_q(mochila):
    suma = sum(mochila)
    return randint(suma+1,suma*10)

def select_r(q):
    try_r = randint(2,q-1)
    while gcd(try_r,q) != 1:
        try_r = randint(2,q)
    return try_r, inverse_mod(try_r,q)

def mochila_camuflada(mochila, q, r):
    return [(r*x)% q for x in mochila]

def codificar(mochila_camuflada, text):
    largo_mochila = len(mochila_camuflada)
    assert largo_mochila % 16 == 0
    
    texto_codificado = text_to_bytes(text,largo_mochila // 16)
    a_codifcar = split2len(texto_codificado, largo_mochila)
    return [sum([b for (a,b) in zip(k,mochila_camuflada) if a=='1']) for k in a_codifcar], texto_codificado


def unpack(n,l):
    i = len(l) -1
    for i in range(len(l)-1,-1,-1):
        if n>0:
            if l[i] <= n:
                n -= l[i]
                res = 1
            else:
                res = 0
            i-=1
        else:
            res = 0
        yield res

def decodificar(chunk, mochila_o, q, i_r):
    message = (chunk * i_r) % q
    int_message = list(unpack(message,mochila_o))[::-1]
    return bytes_to_text(''.join(map(str,int_message)))
    
    


In [192]:
mochi = new_mochila(N=32, jump=5)
q = select_q(mochi)
r, i_r = select_r(q)
mochi_c = mochila_camuflada(mochi, q, r)

texto = 'mi mama me mima'
codifcado, x= codificar(mochi_c, texto)

In [193]:
s=''
for c in codifcado:
    s+=decodificar(c,mochi, q, i_r)
print s

mi mama me mima 
